In [1]:
from imblearn.over_sampling import SMOTE

from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

from collections import Counter

import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder

from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

In [2]:
df1 = pd.read_csv("KnowledgeBase.csv")
df2 = pd.read_csv("KnowledgeBase_majorityvotecluster.csv")
df1.fillna(0,inplace=True)
df2.fillna(0,inplace=True)

In [7]:
df_char_raw1 = df1[[ 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]

df_quality_raw1 = df1[['Completeness', 'Conciseness', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'LabelIssuesPerc', 'FeatureAlgo']]

df_raw1 = df1[['Completeness', 'Conciseness', 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'LabelIssuesPerc','nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]


df_char_bin1 = df1[['cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins','FeatureAlgo']]

df_quality_bin1 = df1[['Completeness_bins',
       'Conciseness_bins', 'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins','FeatureAlgo']]

df_bin1 = df1[['Completeness_bins',
       'Conciseness_bins', 'cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins', 'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins', 'FeatureAlgo']]

In [8]:
df_char_raw2 = df2[[ 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]

df_quality_raw2 = df2[['Completeness', 'Conciseness', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'LabelIssuesPerc', 'FeatureAlgo']]

df_raw2 = df2[['Completeness', 'Conciseness', 'cor.mean', 'cov.mean', 'eigenvalues.mean', 'g_mean.mean', 'h_mean.mean', 'iq_range.mean', 
 'kurtosis.mean', 'mad.mean', 'max.mean', 'mean.mean', 'median.mean', 'min.mean', 'nr_cor_attr', 'nr_norm', 'nr_outliers', 'range.mean', 'sd.mean', 'skewness.mean', 'sparsity.mean',
 't_mean.mean', 'var.mean', 'ClassImbRatio', 'ClassOverlapPerc', 'OutlierPerc', 'attr_to_inst', 'inst_to_attr', 'nr_attr', 'nr_bin', 'nr_inst', 'nr_num','attr_conc.mean', 
 'attr_ent.mean', 'LabelIssuesPerc','nUnique', 'ena', 'snr.mean', 'cEntropy', 'FeatureAlgo']]


df_char_bin2 = df2[['cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins','FeatureAlgo']]

df_quality_bin2 = df2[['Completeness_bins',
       'Conciseness_bins', 'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins','FeatureAlgo']]

df_bin2 = df2[['Completeness_bins',
       'Conciseness_bins', 'cor.mean_bins', 'cov.mean_bins',
       'eigenvalues.mean_bins', 'g_mean.mean_bins', 'h_mean.mean_bins',
       'iq_range.mean_bins', 'kurtosis.mean_bins', 'mad.mean_bins',
       'max.mean_bins', 'mean.mean_bins', 'median.mean_bins', 'min.mean_bins',
       'nr_cor_attr_bins', 'nr_norm_bins', 'nr_outliers_bins',
       'range.mean_bins', 'sd.mean_bins', 'skewness.mean_bins',
       'sparsity.mean_bins', 't_mean.mean_bins', 'var.mean_bins',
       'LabelIssues_bins', 'ClassImbRatio_bins', 'OutlierPerc_bins',
       'ClassOverlapPerc_bins', 'attr_to_inst_bins', 'inst_to_attr_bins',
       'nr_attr_bins', 'nr_inst_bins', 'nr_num_bins', 'nr_bin_bins',
       'attr_conc.mean_bins', 'attr_ent.mean_bins', 'ena_bins',
       'snr.mean_bins', 'cEntropy_bins', 'FeatureAlgo']]

In [9]:
print(df1['FeatureAlgo'].value_counts())
print(df2['FeatureAlgo'].value_counts())

chisquare    92
GR           51
fcbf         16
relief        8
MI            6
Name: FeatureAlgo, dtype: int64
chisquare    86
GR           54
fcbf         17
relief        8
MI            8
Name: FeatureAlgo, dtype: int64


In [11]:
#update df_raw
df_raw1['labels_tuple'] = df_raw1.iloc[:, -1].apply(tuple)

tuple_counts = df_raw1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_raw1[df_raw1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_raw1 = df_filtered

#update df_char_raw
df_char_raw1['labels_tuple'] = df_char_raw1.iloc[:, -1].apply(tuple)

tuple_counts = df_char_raw1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_char_raw1[df_char_raw1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_char_raw1 = df_filtered

#update df_quality_raw
df_quality_raw1['labels_tuple'] = df_quality_raw1.iloc[:, -1].apply(tuple)

tuple_counts = df_quality_raw1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_quality_raw1[df_quality_raw1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_quality_raw1 = df_filtered

In [17]:
#update df_raw
df_raw2['labels_tuple'] = df_raw2.iloc[:, -1].apply(tuple)

tuple_counts = df_raw2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_raw2[df_raw2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_raw2 = df_filtered

#update df_char_raw
df_char_raw2['labels_tuple'] = df_char_raw2.iloc[:, -1].apply(tuple)

tuple_counts = df_char_raw2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_char_raw2[df_char_raw2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_char_raw2 = df_filtered

#update df_quality_raw
df_quality_raw2['labels_tuple'] = df_quality_raw2.iloc[:, -1].apply(tuple)

tuple_counts = df_quality_raw2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_quality_raw2[df_quality_raw2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_quality_raw2 = df_filtered

/home/d19125691/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [18]:
#update df_bin
df_bin1['labels_tuple'] = df_bin1.iloc[:, -1].apply(tuple)

tuple_counts = df_bin1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_bin1[df_bin1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_bin1 = df_filtered

#update df_char_bin
df_char_bin1['labels_tuple'] = df_char_bin1.iloc[:, -1].apply(tuple)

tuple_counts = df_char_bin1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_char_bin1[df_char_bin1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_char_bin1 = df_filtered

#update df_quality_bin
df_quality_bin1['labels_tuple'] = df_quality_bin1.iloc[:, -1].apply(tuple)

tuple_counts = df_quality_bin1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_quality_bin1[df_quality_bin1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_quality_bin1 = df_filtered

In [14]:
#update df_bin
df_bin2['labels_tuple'] = df_bin2.iloc[:, -1].apply(tuple)

tuple_counts = df_bin2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_bin2[df_bin2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_bin2 = df_filtered

#update df_char_bin
df_char_bin2['labels_tuple'] = df_char_bin2.iloc[:, -1].apply(tuple)

tuple_counts = df_char_bin2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_char_bin2[df_char_bin2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_char_bin2 = df_filtered

#update df_quality_bin
df_quality_bin2['labels_tuple'] = df_quality_bin2.iloc[:, -1].apply(tuple)

tuple_counts = df_quality_bin2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_quality_bin2[df_quality_bin2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_quality_bin2 = df_filtered

/home/d19125691/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/d19125691/.local/lib/python3.6/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/home/d19125691/.local/lib/python3.6/site-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

## kNN

In [19]:
def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D

    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)


    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
        
    knn_classifier = KNeighborsClassifier(n_neighbors=4)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets1 = {
    'df_raw': df_raw1,
    'df_char_raw': df_char_raw1,
    'df_quality_raw': df_quality_raw1
}

results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_knn(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************KNN - 0.1_0.06**************\n")
print(comparison_df1)

datasets2 = {
    'df_raw': df_raw2,
    'df_char_raw': df_char_raw2,
    'df_quality_raw': df_quality_raw2
}

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_knn(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************KNN - Majority clustering**************\n")
print(comparison_df2)


**************KNN - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.622399                    0.377601   
df_char_raw                     0.622399                    0.377601   
df_quality_raw                  0.527138                    0.472862   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.608696        0.391304  0.652656  0.608696  0.610317   
df_char_raw         0.608696        0.391304  0.652656  0.608696  0.610317   
df_quality_raw       0.48913         0.51087   0.45243   0.48913  0.457957   

                                                 Confusion Matrix  
df_raw          [[12, 1, 2, 0, 0], [2, 14, 0, 0, 0], [8, 2, 5,...  
df_char_raw     [[12, 1, 2, 0, 0], [2, 14, 0, 0, 0], [8, 2, 5,...  
df_quality_raw  [[11, 1, 2, 0, 1], [2, 11, 1, 2, 0], [9, 1, 1,...  


**************KNN - Majority clustering**************

               Cross-Vali

## Decision trees

In [20]:

def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y)

    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_decision_tree(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************DT - 0.1_0.06**************\n")
print(comparison_df1)

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_decision_tree(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************DT - Majority clustering**************\n")
print(comparison_df2)


**************DT - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.785524                    0.214476   
df_char_raw                     0.744798                    0.255202   
df_quality_raw                  0.622584                    0.377416   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw               0.76087         0.23913   0.77033   0.76087  0.753872   
df_char_raw         0.728261        0.271739  0.744167  0.728261  0.713651   
df_quality_raw      0.586957        0.413043  0.584077  0.586957  0.567055   

                                                 Confusion Matrix  
df_raw          [[10, 1, 3, 1, 0], [0, 14, 0, 2, 0], [8, 0, 7,...  
df_char_raw     [[10, 1, 0, 2, 2], [0, 14, 0, 2, 0], [7, 0, 6,...  
df_quality_raw  [[13, 1, 0, 0, 1], [1, 15, 0, 0, 0], [8, 3, 4,...  


**************DT - Majority clustering**************

               Cross-Valida

## Distance-based

In [22]:
results1 = {}
results2 = {}

metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=33)

for name, df in datasets1.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []

    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        distances = metric_func(X_train, X_test)
        nearest_indices = np.argmin(distances, axis=0)
        predicted_labels = y_train[nearest_indices]
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    
    nearest_indices = np.argmin(distances, axis=0)
    
    predicted_labels = y_train[nearest_indices]
    
    test_accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)

    results1[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }


for name, df in datasets2.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    cv_accuracies = []

    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        distances = metric_func(X_train, X_test)
        nearest_indices = np.argmin(distances, axis=0)
        predicted_labels = y_train[nearest_indices]
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    
    nearest_indices = np.argmin(distances, axis=0)
    
    predicted_labels = y_train[nearest_indices]
    
    test_accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)

    results2[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df1 = pd.DataFrame(results1).transpose()
results_df2 = pd.DataFrame(results2).transpose()

print("**************Distance based - 0.1_0.06**************\n")
print(results_df1)

print("\n\n**************Distance based - Majority clustering**************\n")
print(results_df2)



**************Distance based - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                          0.708696                    0.291304   
df_char_raw                     0.708696                    0.291304   
df_quality_raw                  0.617391                    0.382609   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.717391        0.282609  0.727137  0.717391   0.71314   
df_char_raw         0.717391        0.282609  0.727137  0.717391   0.71314   
df_quality_raw      0.586957        0.413043  0.556289  0.586957  0.567659   

                                                 Confusion Matrix  
df_raw          [[12, 0, 3, 0, 0], [2, 12, 0, 2, 0], [5, 1, 7,...  
df_char_raw     [[12, 0, 3, 0, 0], [2, 12, 0, 2, 0], [5, 1, 7,...  
df_quality_raw  [[12, 0, 2, 1, 0], [0, 12, 1, 3, 0], [3, 3, 4,...  


**************Distance based - Majority clustering**************

   

## Unification

In [23]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    
    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])

        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }


results1 = {}
results2 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_unification(df)

results_df1 = pd.DataFrame(results1).transpose()

for name, df in datasets2.items():
    results2[name] = evaluate_unification(df)

results_df2 = pd.DataFrame(results2).transpose()

print("**************Unification - 0.1_0.06**************\n")
print(results_df1)

print("\n\n**************Unification - Majority clustering**************\n")
print(results_df2)


**************Unification - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_raw                               0.8                         0.2   
df_char_raw                          0.8                         0.2   
df_quality_raw                      0.53                        0.47   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_raw              0.666667        0.333333  0.666667  0.666667  0.666667   
df_char_raw         0.666667        0.333333  0.666667  0.666667  0.666667   
df_quality_raw          0.75            0.25      0.75      0.75      0.75   

                                                 Confusion Matrix  
df_raw                          [[0, 0, 0], [1, 0, 0], [0, 0, 2]]  
df_char_raw                     [[0, 0, 0], [1, 0, 0], [0, 0, 2]]  
df_quality_raw  [[2, 0, 0, 0], [0, 2, 1, 0], [0, 1, 0, 0], [0,...  


**************Unification - Majority clustering**************

         

## Bin data

## kNN

In [24]:
def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    

    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D

    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)


    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
        
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

results1 = {}

datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}

for name, df in datasets1.items():
    results1[name] = evaluate_knn(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************KNN - 0.1_0.06**************\n")
print(comparison_df1)

datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_bin': df_quality_bin2
}

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_knn(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************KNN - Majority clustering**************\n")
print(comparison_df2)

**************KNN - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.660348                    0.339652   
df_char_bin                     0.584043                    0.415957   
df_quality_bin                   0.43221                     0.56779   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin               0.73913         0.26087  0.774245   0.73913  0.719318   
df_char_bin         0.532609        0.467391  0.549597  0.532609  0.509996   
df_quality_bin      0.315217        0.684783  0.263472  0.315217  0.258322   

                                                 Confusion Matrix  
df_bin          [[11, 0, 1, 1, 2], [0, 16, 0, 0, 0], [9, 1, 5,...  
df_char_bin     [[6, 1, 1, 5, 2], [0, 15, 0, 1, 0], [9, 1, 3, ...  
df_quality_bin  [[3, 5, 2, 5, 0], [0, 16, 0, 0, 0], [11, 1, 5,...  


**************KNN - Majority clustering**************

               Cross-Vali

## Decision trees

In [25]:

def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D

    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

results1 = {}

datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}

for name, df in datasets1.items():
    results1[name] = evaluate_decision_tree(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************DT - 0.1_0.06**************\n")
print(comparison_df1)

datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_bin': df_quality_bin2
}

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_decision_tree(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************DT - Majority clustering**************\n")
print(comparison_df2)

**************DT - 0.1_0.06**************

               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.682007                    0.317993   
df_char_bin                     0.587042                    0.412958   
df_quality_bin                  0.500037                    0.499963   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.793478        0.206522  0.792472  0.793478   0.77019   
df_char_bin         0.554348        0.445652  0.627435  0.554348   0.53412   
df_quality_bin      0.456522        0.543478  0.498372  0.456522  0.428068   

                                                 Confusion Matrix  
df_bin          [[12, 0, 1, 0, 2], [0, 16, 0, 0, 0], [7, 1, 6,...  
df_char_bin     [[6, 1, 1, 7, 0], [0, 15, 0, 1, 0], [8, 1, 3, ...  
df_quality_bin  [[3, 3, 2, 2, 5], [0, 15, 0, 0, 1], [11, 0, 5,...  


**************DT - Majority clustering**************

               Cross-Valida

## Distance based

In [28]:
datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}
datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_bin': df_quality_bin2
}
results1 = {}
results2 = {}

metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=33)

for name, df in datasets1.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D
    
  
    cv_accuracies = []

        
    smote = SMOTE(random_state=33)
    

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    
    
    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    
    nearest_indices = np.argmin(distances, axis=0)
    
    predicted_labels = y_train[nearest_indices]
    
    test_accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)

    results1[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }


results_df1 = pd.DataFrame(results1).transpose()


for name, df in datasets2.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D
    
  
    cv_accuracies = []

        
    smote = SMOTE(random_state=33)
    

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    
    
    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    
    nearest_indices = np.argmin(distances, axis=0)
    
    predicted_labels = y_train[nearest_indices]
    
    test_accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)

    results2[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df2 = pd.DataFrame(results2).transpose()

print("***************Distance based - 0.1_0.06*************")
print(results_df1)
print("\n\n***************Distance based - Majority cluster*************")
print(results_df2)



***************Distance based - 0.1_0.06*************
               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.558696                    0.441304   
df_char_bin                     0.352174                    0.647826   
df_quality_bin                  0.258696                    0.741304   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.663043        0.336957  0.644136  0.663043  0.646848   
df_char_bin         0.543478        0.456522  0.633122  0.543478  0.539469   
df_quality_bin      0.445652        0.554348  0.444755  0.445652  0.423903   

                                                 Confusion Matrix  
df_bin          [[10, 0, 4, 1, 0], [0, 15, 1, 0, 0], [10, 1, 3...  
df_char_bin     [[9, 1, 0, 4, 1], [1, 9, 0, 6, 0], [9, 1, 4, 4...  
df_quality_bin  [[3, 2, 2, 5, 3], [0, 9, 2, 5, 0], [5, 1, 4, 7...  


***************Distance based - Majority cluster*************
        

## Unification

In [29]:
def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D
    

    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    
    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])

        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }



datasets1 = {
    'df_bin': df_bin1,
    'df_char_bin': df_char_bin1,
    'df_quality_bin': df_quality_bin1
}
datasets2 = {
    'df_bin': df_bin2,
    'df_char_bin': df_char_bin2,
    'df_quality_bin': df_quality_bin2
}
results1 = {}
results2 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_unification(df)
for name, df in datasets2.items():
    results2[name] = evaluate_unification(df)

results_df1 = pd.DataFrame(results1).transpose()
results_df2 = pd.DataFrame(results2).transpose()

print("***************Unification - 0.1_0.06**********************")
print(results_df1)
print("***************Unification - majority voting**********************")
print(results_df2)


***************Unification - 0.1_0.06**********************
               Cross-Validation Accuracy Cross-Validation Error Rate  \
df_bin                          0.187415                    0.812585   
df_char_bin                     0.214944                    0.785056   
df_quality_bin                  0.191121                    0.808879   

               Test Accuracy Test Error Rate Precision    Recall  F1 Score  \
df_bin              0.192771        0.807229   0.20642  0.192771  0.195181   
df_char_bin         0.131868        0.868132  0.150536  0.131868  0.137932   
df_quality_bin      0.230769        0.769231  0.234008  0.230769  0.221084   

                                                 Confusion Matrix  
df_bin          [[2, 1, 4, 2, 4], [5, 1, 3, 5, 1], [6, 2, 2, 3...  
df_char_bin     [[2, 2, 6, 3, 1], [5, 3, 5, 0, 3], [8, 2, 1, 2...  
df_quality_bin  [[7, 3, 1, 4, 0], [4, 1, 4, 3, 3], [4, 3, 5, 5...  
***************Unification - majority voting**********************

In [33]:
import pandas as pd
df_normalised1 = pd.read_csv("NormalisedDataset_01006.csv")
df_normalised2 = pd.read_csv("NormalisedDataset_majoritycluster.csv")

df_normalised_quality1 = df_normalised1[['Completeness_normalized', 'Conciseness_normalized',  'ClassImbRatio_normalized', 'ClassOverlapPerc_normalized',
       'OutlierPerc_normalized',  'LabelIssuesPerc_normalized', 'FeatureAlgo_normalized']]
df_normalised_char1 = df_normalised1[[ 'cor.mean_normalized', 'cov.mean_normalized',
       'eigenvalues.mean_normalized', 'g_mean.mean_normalized',
       'h_mean.mean_normalized', 'iq_range.mean_normalized',
       'kurtosis.mean_normalized', 'mad.mean_normalized',
       'max.mean_normalized', 'mean.mean_normalized', 'median.mean_normalized',
       'min.mean_normalized', 'nr_cor_attr_normalized', 'nr_norm_normalized',
       'nr_outliers_normalized', 'range.mean_normalized', 'sd.mean_normalized',
       'skewness.mean_normalized', 'sparsity.mean_normalized',
       't_mean.mean_normalized', 'var.mean_normalized',
        'attr_to_inst_normalized',
       'inst_to_attr_normalized', 'nr_attr_normalized', 'nr_bin_normalized',
       'nr_inst_normalized', 'nr_num_normalized', 'attr_conc.mean_normalized',
       'attr_ent.mean_normalized', 'nUnique_normalized', 'ena_normalized', 'snr.mean_normalized',
       'cEntropy_normalized', 'FeatureAlgo_normalized']]
df_normalised_quality2 = df_normalised2[['Completeness_normalized', 'Conciseness_normalized',  'ClassImbRatio_normalized', 'ClassOverlapPerc_normalized',
       'OutlierPerc_normalized',  'LabelIssuesPerc_normalized', 'FeatureAlgo_normalized']]
df_normalised_char2 = df_normalised2[[ 'cor.mean_normalized', 'cov.mean_normalized',
       'eigenvalues.mean_normalized', 'g_mean.mean_normalized',
       'h_mean.mean_normalized', 'iq_range.mean_normalized',
       'kurtosis.mean_normalized', 'mad.mean_normalized',
       'max.mean_normalized', 'mean.mean_normalized', 'median.mean_normalized',
       'min.mean_normalized', 'nr_cor_attr_normalized', 'nr_norm_normalized',
       'nr_outliers_normalized', 'range.mean_normalized', 'sd.mean_normalized',
       'skewness.mean_normalized', 'sparsity.mean_normalized',
       't_mean.mean_normalized', 'var.mean_normalized',
        'attr_to_inst_normalized',
       'inst_to_attr_normalized', 'nr_attr_normalized', 'nr_bin_normalized',
       'nr_inst_normalized', 'nr_num_normalized', 'attr_conc.mean_normalized',
       'attr_ent.mean_normalized', 'nUnique_normalized', 'ena_normalized', 'snr.mean_normalized',
       'cEntropy_normalized', 'FeatureAlgo_normalized']]

In [35]:
#update df_raw
df_normalised1['labels_tuple'] = df_normalised1.iloc[:, -1].apply(tuple)

tuple_counts = df_normalised1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_normalised1[df_normalised1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_normalised1 = df_filtered

#update df_char_raw
df_normalised_char1['labels_tuple'] = df_normalised_char1.iloc[:, -1].apply(tuple)

tuple_counts = df_normalised_char1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_normalised_char1[df_normalised_char1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_normalised_char1 = df_filtered

#update df_quality_raw
df_normalised_quality1['labels_tuple'] = df_normalised_quality1.iloc[:, -1].apply(tuple)

tuple_counts = df_normalised_quality1['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_normalised_quality1[df_normalised_quality1['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_normalised_quality1 = df_filtered


#update df_raw
df_normalised2['labels_tuple'] = df_normalised2.iloc[:, -1].apply(tuple)

tuple_counts = df_normalised2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_normalised2[df_normalised2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_normalised2 = df_filtered

#update df_char_raw
df_normalised_char2['labels_tuple'] = df_normalised_char2.iloc[:, -1].apply(tuple)

tuple_counts = df_normalised_char2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_normalised_char2[df_normalised_char2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_normalised_char2 = df_filtered

#update df_quality_raw
df_normalised_quality2['labels_tuple'] = df_normalised_quality2.iloc[:, -1].apply(tuple)

tuple_counts = df_normalised_quality2['labels_tuple'].value_counts()

valid_tuples = tuple_counts[tuple_counts >= 6].index
df_filtered = df_normalised_quality2[df_normalised_quality2['labels_tuple'].isin(valid_tuples)].copy()
df_filtered = df_filtered.drop('labels_tuple', axis=1)

df_normalised_quality2 = df_filtered

## kNN

In [37]:

def evaluate_knn(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    label_encoder = LabelEncoder()

    for col in X:
        X[col] = label_encoder.fit_transform(X[col])
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D
    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)

    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
        
    knn_classifier = KNeighborsClassifier(n_neighbors=5)
    
    cross_val_scores = cross_val_score(knn_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    knn_classifier.fit(X_train, y_train)
    y_pred = knn_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }


datasets1 = {
    'df_normalised': df_normalised1,
    'df_normalised_char': df_normalised_char1,
    'df_normalised_quality': df_normalised_quality1
}

results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_knn(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************KNN - 0.1_0.06**************\n")
print(comparison_df1)

datasets2 = {
    'df_normalised': df_normalised2,
    'df_normalised_char': df_normalised_char2,
    'df_normalised_quality': df_normalised_quality2
}

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_knn(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************KNN - Majority clustering**************\n")
print(comparison_df2)

**************KNN - 0.1_0.06**************

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.682007                    0.317993   
df_normalised_char                     0.654795                    0.345205   
df_normalised_quality                  0.429397                    0.570603   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.782609        0.217391  0.842689  0.782609   
df_normalised_char         0.728261        0.271739  0.740092  0.728261   
df_normalised_quality      0.543478        0.456522   0.64301  0.543478   

                       F1 Score  \
df_normalised          0.767894   
df_normalised_char     0.700804   
df_normalised_quality  0.545198   

                                                        Confusion Matrix  
df_normalised          [[15, 0, 0, 0, 0], [0, 16, 0, 0, 0], [9, 1, 6,...  
df_normalised_char     [[14, 0, 0, 0, 1], [1, 1

## Decision trees

In [38]:
#df_raw decision
def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D

    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)

    smote = SMOTE(random_state=33)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

results1 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_decision_tree(df)

comparison_df1 = pd.DataFrame(results1).transpose()
print("**************DT - 0.1_0.06**************\n")
print(comparison_df1)

results2 = {}

for name, df in datasets2.items():
    results2[name] = evaluate_decision_tree(df)

comparison_df2 = pd.DataFrame(results2).transpose()
print("\n\n**************DT - Majority clustering**************\n")
print(comparison_df2)

**************DT - 0.1_0.06**************

                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.744724                    0.255276   
df_normalised_char                     0.706627                    0.293373   
df_normalised_quality                  0.434765                    0.565235   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised               0.73913         0.26087  0.725531   0.73913   
df_normalised_char         0.717391        0.282609  0.726119  0.717391   
df_normalised_quality       0.51087         0.48913  0.515554   0.51087   

                       F1 Score  \
df_normalised          0.715615   
df_normalised_char      0.68756   
df_normalised_quality  0.508566   

                                                        Confusion Matrix  
df_normalised          [[12, 0, 1, 2, 0], [0, 16, 0, 0, 0], [7, 0, 5,...  
df_normalised_char     [[11, 0, 0, 3, 1], [0, 15

## Distance-based

In [39]:
datasets1 = {
    'df_normalised': df_normalised1,
    'df_normalised_char': df_normalised_char1,
    'df_normalised_quality': df_normalised_quality1
}

results1 = {}
results2 = {}

datasets2 = {
    'df_normalised': df_normalised2,
    'df_normalised_char': df_normalised_char2,
    'df_normalised_quality': df_normalised_quality2
}

metric_name = 'Euclidean'
metric_func = lambda X_train, X_test: np.sqrt(((X_train.values[:, np.newaxis] - X_test.values) ** 2).sum(axis=2))

kf = KFold(n_splits=5, shuffle=True, random_state=33)

for name, df in datasets1.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D
  
    cv_accuracies = []
        
    smote = SMOTE(random_state=33)
  
    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    
    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    
    nearest_indices = np.argmin(distances, axis=0)
    
    predicted_labels = y_train[nearest_indices]
    
    test_accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)

    results1[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df1 = pd.DataFrame(results1).transpose()

for name, df in datasets1.items():
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D
  
    cv_accuracies = []
        
    smote = SMOTE(random_state=33)
  
    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    
    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        distances = metric_func(X_train, X_test)
        
        nearest_indices = np.argmin(distances, axis=0)
        
        predicted_labels = y_train[nearest_indices]
        
        cv_accuracy = accuracy_score(y_test, predicted_labels)
        
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    
    distances = metric_func(X_train, X_test)
    
    nearest_indices = np.argmin(distances, axis=0)
    
    predicted_labels = y_train[nearest_indices]
    
    test_accuracy = accuracy_score(y_test, predicted_labels)
    precision = precision_score(y_test, predicted_labels, average='weighted', zero_division=0)
    recall = recall_score(y_test, predicted_labels, average='weighted', zero_division=0)
    f1 = f1_score(y_test, predicted_labels, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(y_test, predicted_labels)

    results2[name] = {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }

results_df2 = pd.DataFrame(results2).transpose()


print("***************Distance - 0.1_0.06**********************")
print(results_df1)
print("***************Distance - majority voting**********************")
print(results_df2)



***************Distance - 0.1_0.06**********************
                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.802174                    0.197826   
df_normalised_char                     0.676087                    0.323913   
df_normalised_quality                  0.445652                    0.554348   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.826087        0.173913  0.826094  0.826087   
df_normalised_char          0.73913         0.26087   0.74513   0.73913   
df_normalised_quality           0.5             0.5  0.494753       0.5   

                       F1 Score  \
df_normalised          0.806917   
df_normalised_char     0.724219   
df_normalised_quality  0.488896   

                                                        Confusion Matrix  
df_normalised          [[14, 0, 0, 1, 0], [0, 15, 0, 1, 0], [6, 1, 7,...  
df_normalised_char     [[12, 0, 0,

## Unification

In [41]:

def unify_features(test_sample, train_data, train_labels):
    test_features = test_sample.values
    matching_indices = np.all(train_data == test_features, axis=1)
    if matching_indices.any():
        matching_labels = train_labels[matching_indices]
        return np.unique(matching_labels)
    return ["unknown"]

def evaluate_unification(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
   
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D
    

    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    cv_accuracies = []
    test_accuracies = []
    precisions = []
    recalls = []
    f1_scores = []
    confusion_matrices = []
    
    smote = SMOTE(random_state=33)

    X_resampled, y_resampled = smote.fit_resample(X, y_encoded)
    
    for train_index, test_index in kf.split(X_resampled):
        X_train, X_test = X_resampled.iloc[train_index], X_resampled.iloc[test_index]
        y_train, y_test = y_resampled[train_index], y_resampled[test_index]
        
        predicted_labels = []
        for i in range(X_test.shape[0]):
            label = unify_features(X_test.iloc[i], X_train, y_train)
            predicted_labels.append(label)
        
        predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
        valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        if not valid_indices:
            continue
        
        valid_predictions = np.array([predicted_labels[i] for i in valid_indices])

        valid_truth = np.array([y_test[i] for i in valid_indices])
        
        cv_accuracy = accuracy_score(valid_truth, valid_predictions)
       
        cv_accuracies.append(cv_accuracy)
    
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)
    predicted_labels = []
    for i in range(X_test.shape[0]):
        label = unify_features(X_test.iloc[i], X_train, y_train)
        predicted_labels.append(label)
    predicted_labels = [item for sublist in predicted_labels for item in sublist]
        
    valid_indices = [i for i, label in enumerate(predicted_labels) if label != "unknown" and i < len(y_test)]
        
    valid_predictions = np.array([predicted_labels[i] for i in valid_indices])
    valid_truth = np.array([y_test[i] for i in valid_indices])
        
    precision = precision_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    recall = recall_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    f1 = f1_score(valid_truth, valid_predictions, average='weighted', zero_division=0)
    confusion_matrix_result = confusion_matrix(valid_truth, valid_predictions)    
    test_accuracy = accuracy_score(valid_truth, valid_predictions)

    return {
        'Cross-Validation Accuracy': np.mean(cv_accuracies),
        'Cross-Validation Error Rate': np.mean([1 - acc for acc in cv_accuracies]),
        'Test Accuracy': test_accuracy,
        'Test Error Rate': 1 - test_accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': confusion_matrix_result
    }


results1 = {}
results2 = {}

for name, df in datasets1.items():
    results1[name] = evaluate_unification(df)
for name, df in datasets2.items():
    results2[name] = evaluate_unification(df)

results_df1 = pd.DataFrame(results1).transpose()
results_df2 = pd.DataFrame(results2).transpose()

print("***************Unification - 0.1_0.06**********************")
print(results_df1)
print("***************Unification - majority voting**********************")
print(results_df2)


***************Unification - 0.1_0.06**********************
                      Cross-Validation Accuracy Cross-Validation Error Rate  \
df_normalised                          0.487242                    0.512758   
df_normalised_char                     0.155642                    0.844358   
df_normalised_quality                  0.199483                    0.800517   

                      Test Accuracy Test Error Rate Precision    Recall  \
df_normalised              0.105263        0.894737  0.133705  0.105263   
df_normalised_char         0.130952        0.869048  0.142452  0.130952   
df_normalised_quality      0.181818        0.818182  0.208175  0.181818   

                       F1 Score  \
df_normalised           0.10649   
df_normalised_char     0.135007   
df_normalised_quality  0.190806   

                                                        Confusion Matrix  
df_normalised          [[1, 2, 1, 0, 1], [3, 1, 0, 3, 0], [2, 2, 0, 3...  
df_normalised_char     [[0, 1, 

## Saving the better model

In [6]:
import pickle
from sklearn.tree import export_text

def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  
    
    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
    
    smote = SMOTE(random_state=33)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    rules = export_text(tree_classifier, feature_names=list(X_train.columns))
    print(rules)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'model': tree_classifier,
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_raw': df_raw,
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_decision_tree(df)

with open('df_raw_decision_tree_model.pkl', 'wb') as model_file:
    pickle.dump(results['df_raw']['model'], model_file)

|--- nr_inst <= 335.50
|   |--- cEntropy <= 0.95
|   |   |--- iq_range.mean <= 310.88
|   |   |   |--- cEntropy <= 0.41
|   |   |   |   |--- cor.mean <= 0.24
|   |   |   |   |   |--- class: chisquare
|   |   |   |   |--- cor.mean >  0.24
|   |   |   |   |   |--- class: relief
|   |   |   |--- cEntropy >  0.41
|   |   |   |   |--- attr_to_inst <= 0.36
|   |   |   |   |   |--- inst_to_attr <= 8.72
|   |   |   |   |   |   |--- LabelIssuesPerc <= 0.02
|   |   |   |   |   |   |   |--- class: chisquare
|   |   |   |   |   |   |--- LabelIssuesPerc >  0.02
|   |   |   |   |   |   |   |--- class: GR
|   |   |   |   |   |--- inst_to_attr >  8.72
|   |   |   |   |   |   |--- attr_to_inst <= 0.01
|   |   |   |   |   |   |   |--- class: chisquare
|   |   |   |   |   |   |--- attr_to_inst >  0.01
|   |   |   |   |   |   |   |--- ena <= -40.54
|   |   |   |   |   |   |   |   |--- class: chisquare
|   |   |   |   |   |   |   |--- ena >  -40.54
|   |   |   |   |   |   |   |   |--- OutlierPerc <= 0.03
|

In [24]:
#df_raw decision
def evaluate_decision_tree(df, random_state=33):
    df = df.copy()
    df.fillna(0, inplace=True)
    
    X = df.iloc[:, :-1]
    y = df.iloc[:, -1]
    

    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)  # Ensure y_encoded is 1D

    kf = KFold(n_splits=5, shuffle=True, random_state=random_state)

    smote = SMOTE(random_state=33)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=33)    
    tree_classifier = DecisionTreeClassifier(random_state=random_state)
    
    cross_val_scores = cross_val_score(tree_classifier, X_train, y_train, cv=kf, scoring='accuracy')
    error_rates = 1 - cross_val_scores
    
    tree_classifier.fit(X_train, y_train)
    y_pred = tree_classifier.predict(X_test)
    
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_test, y_pred, average='weighted', zero_division=0)
    
    conf_matrix = confusion_matrix(y_test, y_pred)
    
    return {
        'model': tree_classifier,
        'Cross-Validation Accuracy': cross_val_scores.mean(),
        'Cross-Validation Error Rate': error_rates.mean(),
        'Test Accuracy': accuracy,
        'Test Error Rate': 1 - accuracy,
        'Precision': precision,
        'Recall': recall,
        'F1 Score': f1,
        'Confusion Matrix': conf_matrix
    }

datasets = {
    'df_normalised': df_normalised
}

results = {}

for name, df in datasets.items():
    results[name] = evaluate_decision_tree(df)

with open('df_raw_normalised_decision_tree_model.pkl', 'wb') as model_file:
    pickle.dump(results['df_normalised']['model'], model_file)